In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Volumes/NJ_4TB/Final/GEOMET.csv', skipinitialspace=True, header=0)

In [2]:
pd.unique(df[df['country'].str.contains('Greece')]['iso'])

array(['GRC'], dtype=object)

In [3]:
emdat_iso_countries={ 'KR':'KOR',
                         'MX':'MEX',
                         'EG':'EGY',
                         'JP':'JPN',
                         'NO':'NOR',
                         'CA':'CAN',
                         'SI':'SVN',
                         'DK':'DNK',
                         'TH':'THA',
                         'FI':'FIN',
                         'HU':'HUN',
                         'IS':'ISL',
                         'IN':'IND',
                         'US':'USA',
                         'CZ':['CZE', 'SVK'],
                         'CL':'CHL',
                         'PL':'POL',
                         'LV':'LVA',
                         'KY':'CYM',
                         'PH':'PHL',
                         'DE': 'DEU',#also DD
                         'SG':'SGP',
                         'CH':'CHE',
                         'LT':'LTU',
                         'CY':'CYP',
                         'RU':'RUS',
                         'UA':'UKR',
                         'IT':'ITA',
                         'AU':'AUS',
                         'NZ':'NZL',
                         'BE':'BEL',
                         'QA':'QAT',
                         'MY':'MYS',
                         'BM':'BMU',
                         'AT':'AUT',
                         #'SU':'Soviet Union',
                         'GR':'GRC',
                         'MT':'MLT',
                         'IE':'IRL',
                         'TR':'TUR',
                         'ES':'ESP',
                         'NL':'NLD',
                         'IL':'ISR',
                         'MA':'MAR',
                         'AE':'ARE',
                         'CU':'CUB',
                         'SE':'SWE',
                         'CN':'CHN',
                         'FR':'FRA',
                         'LU':'LUX',
                         'PA':'PAN',
                         'BG':'BGR',
                         'ZA':'ZAF',
                         'TW':'TWN',
                         'AR':'ARG',
                         'BR':'BRA',
                         'GB':'GBR',
                         'SA':'SAU'
                       }

In [4]:
#Creating instrumets
df['country'] = df['iso']
df_filtered = pd.DataFrame()
for country, iso in emdat_iso_countries.items():
    if country == 'CZ':
        df.loc[df['iso'] == 'CZE','country'] = country
        df.loc[df['iso'] == 'SVK','country'] = country
    else:
        df.loc[df['iso'] == iso,'country'] = country
    df_country = df[df['country']==country][['country','year','region', 'income_group', 'lpop', 'polity', 'low', 'middle', 'high', 'damage_cor', 'damage_gdp', 'combi', 'difmonth', 'abs_diff', 'dif_temp', 'drought', 'flood', 'disindex', 'disindexla', 'indexla', 'dindex']]
    df_country = pd.concat([df_country, df_country.rolling(10).sum().add_prefix('r10_')],axis=1)
    df_filtered = pd.concat([df_filtered,df_country])
df_filtered.drop(columns=['r10_year','r10_country'],inplace=True)

df_filtered.drop(df_filtered[df_filtered.country == 'CZ'].index, inplace=True)
df_filtered.drop(df_filtered[df_filtered.country == 'CY'].index, inplace=True)

In [ ]:
df_filtered.to_csv('/Volumes/NJ_4TB/Final/Geomet_instruments.csv', index=False)

In [ ]:
df_filtered

In [ ]:
df_merged.columns

### Population Density

In [5]:
df_pop_raw = pd.read_csv('/Volumes/NJ_4TB/Final/PopDensity.csv', skipinitialspace=True, skiprows=4)
df_pop_raw.drop(columns=["Country Name", "Indicator Name", "Indicator Code", "Unnamed: 65"], inplace=True)

In [6]:
df_pop = df_pop_raw.melt(id_vars=["Country Code"], 
        var_name="year", 
        value_name="population_density")
df_pop.year = pd.to_numeric(df_pop.year)
df_pop['country'] = df_pop['Country Code']
for country, iso in emdat_iso_countries.items():
    if country == 'CZ':
        df_pop.loc[df_pop['Country Code'] == 'CZE','country'] = country
        df_pop.loc[df_pop['Country Code'] == 'SVK','country'] = country
    else:
        df_pop.loc[df_pop['Country Code'] == iso,'country'] = country
df_pop.drop(columns=["Country Code"], inplace=True)


### Adding End user Energy Prices

In [ ]:
# df_energy_raw = pd.read_csv('/Volumes/NJ_4TB/Final/EndUsePricesTaxes_USD_IEA_CoalGas.csv', skipinitialspace=True)
# df_energy_raw.drop([0,1,3],inplace=True)
# df_energy_raw.drop(columns=["Unnamed: 2"],inplace=True)
# df_energy_raw.head()

### Combine GEOMET with PopDen

In [7]:
df_filtered = pd.merge(df_filtered, df_pop, on=['country', 'year'], how='left')

## Adding Instruments

In [ ]:
#Floods
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/Sum_4_Flood_panel_combined.csv', skipinitialspace=True, header=0)
df_merged = pd.merge(df1, df_filtered,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/4_Flood_panel_combined_instruments.csv',index=False)
del df1

In [ ]:
#Drought
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/Sum_7_Drought_panel_combined.csv', skipinitialspace=True, header=0)
df_merged = pd.merge(df1, df_filtered,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/7_Drought_panel_combined_instruments.csv',index=False)
del df1

In [ ]:
#Storm
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/Sum_3_Storm_panel_combined.csv', skipinitialspace=True, header=0)
df_merged = pd.merge(df1, df_filtered,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/3_Storm_panel_combined_instruments.csv',index=False)
del df1

In [ ]:
#Extreme Temperature
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/Sum_1_ExTemp_panel_combined.csv', skipinitialspace=True, header=0)
df_merged = pd.merge(df1, df_filtered,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/1_ExTemp_panel_combined_instruments.csv',index=False)
del df1

### Updating polity

In [1]:
#processed_controls file has been updated with polity2 in place of polity
import pandas as pd
import numpy as np
df_controls = pd.read_csv('/Volumes/NJ_4TB/Final/controls_processed.csv', skipinitialspace=True, header=0)
df_polity = df_controls[["country", "year", "polity"]]

In [2]:
##Floods
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/4_Flood_panel_combined_instruments.csv', skipinitialspace=True, header=0)
df1 = df1.drop(columns=['polity_y'])
df_merged = pd.merge(df1, df_polity,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/4_Flood_panel_combined_instruments_updated.csv',index=False)
del df1

In [3]:
##Drought
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/7_Drought_panel_combined_instruments.csv', skipinitialspace=True, header=0)
df1 = df1.drop(columns=['polity_y'])
df_merged = pd.merge(df1, df_polity,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/7_Drought_panel_combined_instruments_updated.csv',index=False)
del df1

In [4]:
##Storm
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/3_Storm_panel_combined_instruments.csv', skipinitialspace=True, header=0)
df1 = df1.drop(columns=['polity_y'])
df_merged = pd.merge(df1, df_polity,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/3_Storm_panel_combined_instruments_updated.csv',index=False)
#del df1

In [5]:
##Extreme Temperature
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/1_ExTemp_panel_combined_instruments.csv', skipinitialspace=True, header=0)
df1 = df1.drop(columns=['polity_y'])
df_merged = pd.merge(df1, df_polity,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/1_ExTemp_panel_combined_instruments_updated.csv',index=False)
del df1

### Combining all Disasters

In [8]:
fields = ['4_Flood_K_stock', '4_Flood_granted_family_ge_2_count', '4_Flood_granted_count', '4_Flood_total_count']
df1 = pd.read_csv('/Volumes/NJ_4TB/Final/Sum_4_Flood_panel_combined.csv', skipinitialspace=True, header=0, usecols=lambda x: x != fields)
df_merged = pd.merge(df1, df_filtered,on=['country','year'], how = 'inner')
df_merged.to_csv('/Volumes/NJ_4TB/Final/All_Disasters_panel_combined_instruments.csv',index=False)